# Análise Comparativa de Modelos

### Visão Geral

Neste estudo, o objetivo é prever tendências no mercado automotivo com base em características históricas de veículos fabricados entre os anos de 1970 a 2024. O conjunto de dados utilizado, intitulado “90,000+ Cars Data From 1970 to 2024” e disponível na plataforma Kaggle, foi selecionado para fins de análise exploratória e modelagem preditiva, oferecendo uma visão detalhada da evolução tecnológica, econômica e social do setor automotivo ao longo das décadas.

### Configuração do ambiente

In [54]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split, ShuffleSplit, KFold, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error

In [55]:
df = pd.read_csv('../data/raw/CarsData.csv')
df.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,Manufacturer
0,I10,2017,7495,Manual,11630,Petrol,145,60.1,1.0,hyundi
1,Polo,2017,10989,Manual,9200,Petrol,145,58.9,1.0,volkswagen
2,2 Series,2019,27990,Semi-Auto,1614,Diesel,145,49.6,2.0,BMW
3,Yeti Outdoor,2017,12495,Manual,30960,Diesel,150,62.8,2.0,skoda
4,Fiesta,2017,7999,Manual,19353,Petrol,125,54.3,1.2,ford


In [56]:
df_dc = pd.read_csv('../data/external/DictionaryCarsData.csv')
df_dc.head()

,Variável,Descrição,Tipo,Subtipo
0,Model,O modelo do carro,Qualitativa,Nominal
1,Year,O ano de fabricação do carro,Quantitativa,Discreta
2,Price,O preço do carro,Quantitativa,Continua
3,Transmission,O tipo de transmissão usado no carro,Qualitativa,Nominal
4,Mileage,A quilometragem do carro,Quantitativa,Continua


### Preparação de Dados

In [69]:
nominal_variables = ['model', 'transmission', 'fuelType', 'Manufacturer']
ordinal_variables = []
continuous_variables = ['price', 'mileage', 'engineSize']
discrete_variables = ['year', 'tax']


In [70]:
# Dividindo os dados entre X (características) e y (alvo 'mpg')
X = df.drop(columns='mpg', axis=1)  # Removendo a coluna 'mpg' para deixar as características
y = df['mpg']  # Alvo sendo a coluna 'mpg'

In [71]:
# Verificando as colunas de X
print(X.columns)


Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax',
       'engineSize', 'Manufacturer'],
      dtype='object')


### Pipeline de pré-processamento de dados

In [72]:
nominal_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')), 
    ("encoding", OneHotEncoder(sparse_output=False, handle_unknown='infrequent_if_exist')),
])

ordinal_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='median')),
    ("encoding", OrdinalEncoder()),
])

continuous_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='mean')),
    ("normalization", StandardScaler()),
])

discrete_preprocessor = Pipeline(steps=[
    ("missing", KNNImputer()),
    ("normalization", StandardScaler()),
])


In [73]:
preprocessor = ColumnTransformer([
    ("nominal", nominal_preprocessor, nominal_variables), 
    ("ordinal", ordinal_preprocessor, ordinal_variables), 
    ("continuous", continuous_preprocessor, continuous_variables), 
    ("discrete", discrete_preprocessor, discrete_variables), 
])


### Modelos

In [74]:
models = [DummyRegressor(strategy='mean'), LinearRegression(), KNeighborsRegressor(n_neighbors=5), SVR(), RandomForestRegressor(random_state=42)]
metrics = [
    'neg_mean_absolute_error',
    'neg_mean_squared_error',
    'neg_mean_absolute_percentage_error',
    'r2',
]
monte_carlo = ShuffleSplit(n_splits=10, test_size=.2, random_state=42)

In [ ]:
results_total = None
for model in models:
    model_name = model.__class__.__name__
    print(f"rodando para o modelo: {model_name}")
    approach = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model),
    ])

    scores = cross_validate(
        approach, X, y,
        scoring=metrics, 
        cv=monte_carlo,
        return_train_score=True
    )
    results_model = pd.DataFrame(scores)
    results_model['model'] = model_name
    
    if results_total is None:
        results_total = results_model
    else:
        results_total = pd.concat([results_total, results_model], ignore_index=True)

rodando para o modelo: DummyRegressor
rodando para o modelo: LinearRegression
rodando para o modelo: KNeighborsRegressor
rodando para o modelo: SVR


In [ ]:
results_total.groupby('model').agg(['mean', 'std']).T